
# Fuel Flow Calculation

Example calculation of aircraft fuel flow in descent


In [ ]:
from math import sin

import matplotlib.pyplot as plt
import numpy as np

from pyBADA import atmosphere as atm
from pyBADA import constants as const
from pyBADA import conversions as conv
from pyBADA import utils
from pyBADA.bada3 import Bada3Aircraft
from pyBADA.bada4 import Bada4Aircraft

# create an aircraft
ACList = [
    Bada4Aircraft(badaVersion="DUMMY", acName="Dummy-TWIN"),
    Bada3Aircraft(badaVersion="DUMMY", acName="J2M"),
]

# deviation from ISA temperature
deltaTemp = 0

# definition of altitude range
fl_array = np.arange(0, 401, 10)
altitude_array = conv.ft2m(fl_array * 100)

# --- collect per-aircraft series here ---
series = []  # list of dicts: {"label": str, "alt_ft": list[float], "ff": list[float]}

for AC in ACList:
    # define aircraft mass - here as reference mass
    mass = AC.MREF

    # get the original speed schedule for descent for this aircraft
    [Vdes1, Vdes2, Mdes] = AC.flightEnvelope.getSpeedSchedule(phase="Descent")

    # crossover altitude
    crossAlt = atm.crossOver(cas=Vdes2, Mach=Mdes)

    label = f"({AC.BADAFamilyName}) {AC.acName.strip('_')} (BADA {getattr(AC, 'BADAVersion')}) {mass} kg"

    alt_ft_vals = []
    ff_vals = []

    print(
        f"\n=== {AC.__class__.__name__}  |  BADA {AC.BADAVersion}  |  Mass: {mass:.0f} kg ==="
    )
    print(
        f"{'FL':>4}  {'Alt(ft)':>8}  {'Cfg':>3}  {'M':>6}  {'FF (kg/s)':>10}"
    )
    print("-" * 39)

    for alt in altitude_array:
        # atmosphere properties
        theta, delta, sigma = atm.atmosphereProperties(
            h=alt, DeltaTemp=deltaTemp
        )

        # determine the speed acording to BADA ARPM
        [cas, speedUpdated] = AC.ARPM.descentSpeed(
            h=alt, mass=mass, theta=theta, delta=delta, DeltaTemp=deltaTemp
        )
        # general speed conversion
        [M, CAS, TAS] = atm.convertSpeed(
            v=conv.ms2kt(cas),
            speedType="CAS",
            theta=theta,
            delta=delta,
            sigma=sigma,
        )

        # determine the aerodynamic configuration if necesary
        config = AC.flightEnvelope.getConfig(
            h=alt, phase="Descent", v=CAS, mass=mass,DeltaTemp=deltaTemp
        )

        # calculate Energy Share Factor depending if aircraft is flying constant M or CAS (based on crossover altitude)
        if alt < crossAlt:
            ESF = AC.esf(
                h=alt, flightEvolution="constCAS", M=M, DeltaTemp=deltaTemp
            )
        else:
            ESF = AC.esf(
                h=alt, flightEvolution="constM", M=M, DeltaTemp=deltaTemp
            )

        # =====
        # BADA4
        # =====
        if AC.BADAFamily.BADA4:
            # =================================================================================
            # for altitudes where aircraft descends on 3degree slope in AP and LD configuration
            # =================================================================================
            if config == "AP" or config == "LD":
                gamma = -3.0
                temp_const = (theta * const.temp_0) / (
                    theta * const.temp_0 - deltaTemp
                )
                ROCD_gamma = sin(conv.deg2rad(gamma)) * TAS * (1 / temp_const)

                n = 1.0
                [HLid, LG] = AC.flightEnvelope.getAeroConfig(config=config)
                CL = AC.CL(M=M, delta=delta, mass=mass, nz=n)
                CD = AC.CD(M=M, CL=CL, HLid=HLid, LG=LG)
                Drag = AC.D(M=M, delta=delta, CD=CD)
                Thrust = (ROCD_gamma * mass * const.g) * temp_const / (
                    ESF * TAS
                ) + Drag
                CT = AC.CT(Thrust=Thrust, delta=delta)
                ff = AC.ff(
                    CT=CT, delta=delta, theta=theta, M=M, DeltaTemp=deltaTemp
                )

            # =============================================================
            # for altitudes where aircraft descends in IDLE engine settings
            # =============================================================
            else:
                ff = AC.ff(
                    rating="LIDL",
                    delta=delta,
                    theta=theta,
                    M=M,
                   DeltaTemp=deltaTemp,
                )  # [kg/s]

        # =====
        # BADA3
        # =====
        elif AC.BADAFamily.BADA3:
            adaptedThrust = False
            if AC.engineType in ("PISTON", "ELECTRIC"):
                # PISTON  and ELECTRIC uses LIDL throughout the whole descent phase
                config = "CR"
                adaptedThrust = True

            # =================================================================================
            # for altitudes where aircraft descends on 3degree slope in AP and LD configuration
            # =================================================================================
            if config in ("AP", "LD"):
                gamma = -3.0
                temp_const = (theta * const.temp_0) / (
                    theta * const.temp_0 - deltaTemp
                )
                ROCD_gamma = sin(conv.deg2rad(gamma)) * TAS * (1 / temp_const)

                n = 1.0
                CL = AC.CL(sigma=sigma, mass=mass, tas=TAS, nz=n)
                CD = AC.CD(CL=CL, config=config)
                Drag = AC.D(sigma=sigma, tas=TAS, CD=CD)
                Thrust = AC.Thrust(
                    rating="ADAPTED",
                    v=TAS,
                    config=config,
                    h=alt,
                    ROCD=ROCD_gamma,
                    mass=mass,
                    acc=0,
                    deltaTemp=deltaTemp,
                    Drag=Drag,
                )
                ff = AC.ff(
                    flightPhase="Descent",
                    v=TAS,
                    h=alt,
                    T=Thrust,
                    config=config,
                    adapted=adaptedThrust,
                )

            # =============================================================
            # for altitudes where aircraft descends in IDLE engine settings
            # =============================================================
            else:
                ff = AC.ff(
                    v=TAS,
                    h=alt,
                    T=Thrust,
                    flightPhase="Descent",
                    config=config,
                )

        fl = int(utils.proper_round(conv.m2ft(alt) / 100))
        alt_ft = conv.m2ft(alt)
        print(f"{fl:>4d}  {alt_ft:>8.0f}  {config:>3}  {M:>6.3f}  {ff:>10.6f}")

        alt_ft_vals.append(alt_ft)
        ff_vals.append(float(ff))

    series.append(
        {
            "label": label,
            "alt_ft": np.array(alt_ft_vals),
            "ff": np.array(ff_vals),
        }
    )

# --- PLOT: Fuel flow vs Altitude for all aircraft ---
plt.figure(figsize=(8, 5))
for s in series:
    plt.plot(s["alt_ft"], s["ff"], label=s["label"])

plt.xlabel("Altitude (ft)")
plt.ylabel("Fuel Flow (kg/s)")
plt.title("Descent Fuel Flow vs Altitude")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()


=== Bada4Aircraft  |  BADA DUMMY  |  Mass: 65000 kg ===
  FL   Alt(ft)  Cfg       M   FF (kg/s)
---------------------------------------
   0         0   LD   0.201    0.852485
  10      1000   LD   0.212    0.822006
  20      2000   AP   0.279    0.378778
  30      3000   CR   0.351    0.172248
  40      4000   CR   0.357    0.166989
  50      5000   CR   0.364    0.162016
  60      6000   CR   0.420    0.151997
  70      7000   CR   0.428    0.148105
  80      8000   CR   0.436    0.144442
  90      9000   CR   0.444    0.140994
 100     10000   CR   0.541    0.134151
 110     11000   CR   0.551    0.131689
 120     12000   CR   0.561    0.129358
 130     13000   CR   0.571    0.127143
 140     14000   CR   0.582    0.125033
 150     15000   CR   0.593    0.123014
 160     16000   CR   0.604    0.121075
 170     17000   CR   0.615    0.119203
 180     18000   CR   0.627    0.117385
 190     19000   CR   0.639    0.115608
 200     20000   CR   0.651    0.113860
 210     21000   CR   0

TypeError: Missing v argument

In [ ]:
from pyBADA import atmosphere as atm
from pyBADA import constants as const
from pyBADA import conversions as conv
from pyBADA import utils
from pyBADA.bada3 import Bada3Aircraft
from pyBADA.bada4 import Bada4Aircraft

# create an aircraft
ACList = [
    Bada4Aircraft(badaVersion="DUMMY", acName="Dummy-TWIN"),
    Bada3Aircraft(badaVersion="DUMMY", acName="J2M"),
]

In [ ]:

# Fixed BADA configuration
bada_version = "4.2"
filepath = "/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models"

# Aircraft configuration
ac_model = "A320-232"  # Aircraft model
aircraft_mass = 64000  # Aircraft mass (kg)
DeltaTemp = 0  # Temperature deviation (ISA+0)

# Descent speed configuration
constant_cas = None  # If fixed CAS needed, set value (kt), otherwise None
descent_mach = 0.78  # Descent Mach number
crossover_fl = 290  # Mach/CAS crossover flight level
high_cas = 280  # High altitude CAS (kt)
low_cas_fl = 100  # Low/high CAS crossover flight level
low_cas = 250  # Low altitude CAS (kt)

# Initialize aircraft model
AC = Bada4Aircraft(
    badaVersion=bada_version,
    acName=ac_model,
    filePath=filepath,
)

# Create BADA4 performance computation object
bada4 = BADA4(AC)


In [ ]:
from pyBADA import atmosphere as atm
from pyBADA import constants as const
from pyBADA import conversions as conv
from pyBADA.bada3 import Bada3Aircraft, BADA3, Airplane

AC = Bada3Aircraft(
    badaVersion=3.15,
    acName="A320",  # 使用不带下划线的飞机型号代码
    filePath="/home/longqin/Downloads/3.15/BADA_3.15_Release_files",  # 指向包含.APF/.OPF/.PTD/.PTF文件的目录
)
bada3 = BADA3(AC)


altitude_ft = 37000
altitude_m = conv.ft2m(altitude_ft)
DeltaTemp = 0 
theta_val, delta_val, sigma_val = atm.atmosphereProperties(altitude_m, DeltaTemp)


In [21]:
aircraft_mass = 60000  # Aircraft mass (kg)
mach =  0.76
tas = atm.mach2Tas(mach,theta_val )

CL = bada3.CL(sigma = sigma_val, mass = aircraft_mass , tas = tas, nz= 1 )
print(CL)
CD = bada3.CD (CL= CL, config='CR' )

print(CD)

D = bada3.D(sigma= sigma_val, tas = tas, CD= CD )

ff_gen = bada3.ff(h= altitude_m , v = tas, T = D, config= 'CR',flightPhase= 'Cruise' )
print(ff_gen)

0.5479532147481114
0.03828658704975734
0.5753378340658226
